In [30]:
credit <- read.csv("credit.csv", stringsAsFactors = TRUE)

str(credit)

'data.frame':	1000 obs. of  17 variables:
 $ checking_balance    : Factor w/ 4 levels "< 0 DM","> 200 DM",..: 1 3 4 1 1 4 4 3 4 3 ...
 $ months_loan_duration: int  6 48 12 42 24 36 24 36 12 30 ...
 $ credit_history      : Factor w/ 5 levels "critical","good",..: 1 2 1 2 4 2 2 2 2 1 ...
 $ purpose             : Factor w/ 6 levels "business","car",..: 5 5 4 5 2 4 5 2 5 2 ...
 $ amount              : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ savings_balance     : Factor w/ 5 levels "< 100 DM","> 1000 DM",..: 5 1 1 1 1 5 4 1 2 1 ...
 $ employment_duration : Factor w/ 5 levels "< 1 year","> 7 years",..: 2 3 4 4 3 3 2 3 4 5 ...
 $ percent_of_income   : int  4 2 2 2 3 2 3 2 2 4 ...
 $ years_at_residence  : int  4 2 3 4 4 4 4 2 4 2 ...
 $ age                 : int  67 22 49 45 53 35 53 35 61 28 ...
 $ other_credit        : Factor w/ 3 levels "bank","none",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ housing             : Factor w/ 3 levels "other","own",..: 2 2 2 1 1 1 2 3 2 2 ...
 $ exi

In [31]:
table(credit$checking_balance)

table(credit$savings_balance)


    < 0 DM   > 200 DM 1 - 200 DM    unknown 
       274         63        269        394 


     < 100 DM     > 1000 DM  100 - 500 DM 500 - 1000 DM       unknown 
          603            48           103            63           183 

In [32]:
summary(credit$months_loan_duration)

summary(credit$amount)



   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    4.0    12.0    18.0    20.9    24.0    72.0 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 

In [33]:
table(credit$default)



 no yes 
700 300 

In [34]:
set.seed(9829)

train_sample <- sample(1000, 900)

str(train_sample)

credit_train <- credit[train_sample, ]

credit_test <- credit[-train_sample, ]

prop.table(table(credit_train$default))

prop.table(table(credit_test$default))



 int [1:900] 653 866 119 152 6 617 250 343 367 138 ...



       no       yes 
0.7055556 0.2944444 


  no  yes 
0.65 0.35 

In [35]:
library(C50)

credit_model <- C5.0(default ~., data = credit_train)

credit_model


Call:
C5.0.formula(formula = default ~ ., data = credit_train)

Classification Tree
Number of samples: 900 
Number of predictors: 16 

Tree size: 67 

Non-standard options: attempt to group attributes


In [36]:
summary(credit_model)




Call:
C5.0.formula(formula = default ~ ., data = credit_train)


C5.0 [Release 2.07 GPL Edition]  	Wed Sep 25 19:47:07 2024
-------------------------------

Class specified by attribute `outcome'

Read 900 cases (17 attributes) from undefined.data

Decision tree:

checking_balance in {> 200 DM,unknown}: no (415/55)
checking_balance in {< 0 DM,1 - 200 DM}:
:...credit_history in {perfect,very good}: yes (59/16)
    credit_history in {critical,good,poor}:
    :...months_loan_duration > 27:
        :...dependents > 1:
        :   :...age <= 45: no (12/2)
        :   :   age > 45: yes (2)
        :   dependents <= 1:
        :   :...savings_balance = > 1000 DM: no (2/1)
        :       savings_balance = 500 - 1000 DM: yes (1)
        :       savings_balance = 100 - 500 DM:
        :       :...credit_history = critical: no (1)
        :       :   credit_history = good: yes (7)
        :       :   credit_history = poor:
        :       :   :...existing_loans_count <= 1: no (3)
        :     

In [37]:
credit_pred <- predict(credit_model, credit_test)

library(gmodels)

CrossTable(credit_test$default, credit_pred, prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE, dnn = c('actual default', 'predicted default'))




 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        56 |         9 |        65 | 
               |     0.560 |     0.090 |           | 
---------------|-----------|-----------|-----------|
           yes |        24 |        11 |        35 | 
               |     0.240 |     0.110 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        80 |        20 |       100 | 
---------------|-----------|-----------|-----------|

 


In [38]:
credit_boost10 <- C5.0(default ~., data = credit_train, trials = 10)

credit_boost10


Call:
C5.0.formula(formula = default ~ ., data = credit_train, trials = 10)

Classification Tree
Number of samples: 900 
Number of predictors: 16 

Number of boosting iterations: 10 
Average tree size: 57.3 

Non-standard options: attempt to group attributes


In [39]:
summary(credit_boost10)

credit_boost_pred10 <- predict(credit_boost10, credit_test)

CrossTable(credit_test$default, credit_boost_pred10, prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE, dnn = c('actual default', 'predicted default'))




Call:
C5.0.formula(formula = default ~ ., data = credit_train, trials = 10)


C5.0 [Release 2.07 GPL Edition]  	Wed Sep 25 19:47:07 2024
-------------------------------

Class specified by attribute `outcome'

Read 900 cases (17 attributes) from undefined.data

-----  Trial 0:  -----

Decision tree:

checking_balance in {> 200 DM,unknown}: no (415/55)
checking_balance in {< 0 DM,1 - 200 DM}:
:...credit_history in {perfect,very good}: yes (59/16)
    credit_history in {critical,good,poor}:
    :...months_loan_duration > 27:
        :...dependents > 1:
        :   :...age <= 45: no (12/2)
        :   :   age > 45: yes (2)
        :   dependents <= 1:
        :   :...savings_balance = > 1000 DM: no (2/1)
        :       savings_balance = 500 - 1000 DM: yes (1)
        :       savings_balance = 100 - 500 DM:
        :       :...credit_history = critical: no (1)
        :       :   credit_history = good: yes (7)
        :       :   credit_history = poor:
        :       :   :...existing_lo


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        58 |         7 |        65 | 
               |     0.580 |     0.070 |           | 
---------------|-----------|-----------|-----------|
           yes |        19 |        16 |        35 | 
               |     0.190 |     0.160 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        77 |        23 |       100 | 
---------------|-----------|-----------|-----------|

 


In [40]:
matrix_dimensions <- list(c("no", "yes"), c("no", "yes"))

names(matrix_dimensions) <- c("predicted", "actual")

matrix_dimensions

error_cost <- matrix(c(0, 1, 4, 0), nrow = 2, dimnames = matrix_dimensions)

error_cost

$predicted
[1] "no"  "yes"

$actual
[1] "no"  "yes"

,no,yes
no,0,4
yes,1,0


In [41]:
credit_cost <- C5.0(default ~. , data = credit_train, costs = error_cost)

credit_cost_pred <- predict(credit_cost, credit_test)

CrossTable(credit_test$default, credit_cost_pred,
    prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE, dnn = c('actual default', 'predicted default'))




 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        34 |        31 |        65 | 
               |     0.340 |     0.310 |           | 
---------------|-----------|-----------|-----------|
           yes |         5 |        30 |        35 | 
               |     0.050 |     0.300 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        39 |        61 |       100 | 
---------------|-----------|-----------|-----------|

 
